## Model Monitoring Setup
Data Capture, Baseline Generation, Deployment Preparation for Model Quality Monitoring

In [18]:
# Imports & Setup
import boto3
import pandas as pd
import numpy as np
import json
import botocore
from botocore.exceptions import ClientError

import sagemaker
from sagemaker import get_execution_role, Session
from sagemaker.model_monitor import DataCaptureConfig, DefaultModelMonitor
from sagemaker.sklearn.model import SKLearnModel

In [19]:
# Initialize session and role
session = Session()
role = get_execution_role()
region = session.boto_region_name

# Define bucket and paths
bucket = 'sagemaker-us-east-1-531690656306'
prefix = 'cardio_data'
s3_client = boto3.client('s3', region_name=region)
sagemaker_client = boto3.client('sagemaker', region_name=region)

print("SageMaker session initialized")

SageMaker session initialized


In [20]:
# Load your existing engineered dataset
df = pd.read_csv('cardio_engineered.csv')

# Check data
print(df.head())

# Save cleaned CSV
clean_file = 'cardio_engineered_clean.csv'
df.to_csv(clean_file, index=False, encoding='utf-8-sig')

# Upload cleaned CSV to S3
s3_client.upload_file(clean_file, bucket, f'{prefix}/cardio_engineered_clean.csv')
"Cleaned dataset uploaded to S3"

   age  gender  height_ft  weight_lbs  systolic_bp  diastolic_bp  cholesterol  \
0   50       2       5.51      136.69          110            80            1   
1   55       1       5.12      187.39          140            90            3   
2   51       1       5.41      141.10          130            70            3   
3   48       2       5.54      180.78          150           100            1   
4   47       1       5.12      123.46          100            60            1   

   gluc  smoke  alco  ...  cholesterol_label  pulse_pressure  chol_bmi_ratio  \
0     1      0     0  ...             Normal              30            4.55   
1     1      0     0  ...  Well Above Normal              50            8.60   
2     1      0     0  ...  Well Above Normal              60           12.74   
3     1      0     0  ...             Normal              50            3.48   
4     1      0     0  ...             Normal              40            4.35   

  height_in age_years  is_hypert

'Cleaned dataset uploaded to S3'

In [21]:
# Model and inference script
model_artifact = f's3://{bucket}/model/logistic_model.tar.gz'
entry_point = 'inference.py'
endpoint_name = 'cardio-logistic-monitor-endpoint'

In [22]:
# Create model object
sklearn_model = SKLearnModel(
    model_data=model_artifact,
    role=role,
    entry_point=entry_point,
    framework_version='0.23-1',
    sagemaker_session=session
)

# Enable full data capture
data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=f's3://{bucket}/data-capture',
    capture_options=['Request', 'Response']
)

In [23]:
# Deploy only if endpoint doesn't exist
def deploy_if_not_exists(model, endpoint_name, instance_type, data_capture_config):
    try:
        sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
        print(f"Endpoint '{endpoint_name}' already exists. Skipping deployment.")
    except sagemaker_client.exceptions.ClientError as e:
        if 'Could not find endpoint' in str(e):
            model.deploy(
                initial_instance_count=1,
                instance_type=instance_type,
                endpoint_name=endpoint_name,
                data_capture_config=data_capture_config
            )
            print(f"Deployed endpoint '{endpoint_name}' successfully.")
        else:
            raise

# Call deployment function
deploy_if_not_exists(
    model=sklearn_model,
    endpoint_name=endpoint_name,
    instance_type='ml.m5.xlarge',
    data_capture_config=data_capture_config
)

Endpoint 'cardio-logistic-monitor-endpoint' already exists. Skipping deployment.


In [24]:
# Create model monitor object
monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
    sagemaker_session=session
)

# Define baseline input/output locations
baseline_data_uri = f's3://{bucket}/{prefix}/cardio_engineered_clean.csv'
baseline_results_uri = f's3://{bucket}/{prefix}/baseline-results'

# Suggest baseline job
baseline_job = monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri,
    dataset_format={'csv': {'header': True}},
    output_s3_uri=baseline_results_uri,
    wait=True
)

print("Baseline generation complete")


Job Name:  baseline-suggestion-job-2025-06-11-03-23-46-655
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-531690656306/cardio_data/cardio_engineered_clean.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-531690656306/cardio_data/baseline-results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
................2025-06-11 03:26:21.434222: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-06-11 03:26:21.434252: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart

Full model monitoring pipeline has been prepared by first creating the SKLearn model object, which wraps the trained logistic regression model and inference script so SageMaker can use it. Then I enable data capture, which collects real-time input data and predictions as the model serves traffic, creating a record of live inference data. After that, I deploy the model as an endpoint to make it accessible for inference while capturing data. Once deployed, I generate a baseline using the fully cleaned and preprocessed training dataset; this allows SageMaker to compute statistics and constraints that define the model’s expected data distributions. These baseline statistics become the reference point that future data will be compared against to detect drift or anomalies. This full setup ensures that once monitoring jobs are configured, SageMaker can automatically evaluate incoming data for shifts that may impact model accuracy or stability.

* Calculates <b>statistics</b> (distribution, min, max, mean, std, percentiles, etc.)
* Generates <b>constraints</b> (rules/thresholds learned from your dataset, e.g., feature X must be within certain boundaries)

### Check Constraint Files Exist

In [25]:
# Initialize S3 client
s3 = boto3.client('s3')

# Your bucket and baseline folder
bucket = 'sagemaker-us-east-1-531690656306'
baseline_prefix = 'cardio_data/baseline-results/'

# List objects in the baseline folder
response = s3.list_objects_v2(Bucket=bucket, Prefix=baseline_prefix)

# Check and print files
if 'Contents' in response:
    print("Baseline statistics and constraints found:")
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("No baseline files found.")

Baseline statistics and constraints found:
cardio_data/baseline-results/constraints.json
cardio_data/baseline-results/statistics.json


In [26]:
# Search for all saved baseline results
s3 = boto3.client('s3')
bucket = 'sagemaker-us-east-1-531690656306'

# List all files with 'baseline' in the key
response = s3.list_objects_v2(Bucket=bucket)

print("Files containing 'baseline':\n")

if 'Contents' in response:
    for obj in response['Contents']:
        key = obj['Key']
        if 'baseline' in key.lower():
            print(f"{key}")
else:
    print("No objects found in the bucket.")

Files containing 'baseline':

cardio_data/baseline-results/constraints.json
cardio_data/baseline-results/statistics.json
cardio_project/cardio_logistic_baseline.ipynb
cardio_project/cardio_logistic_baseline_complete.ipynb
cardio_project/cardio_logistic_baseline_v2.ipynb


In [27]:
# Save Notebook to S3
!aws s3 cp cardio_model_monitoring.ipynb s3://sagemaker-us-east-1-531690656306/cardio_project/cardio_model_monitoring.ipynb
!aws s3 cp cardio_engineered_clean.csv s3://sagemaker-us-east-1-531690656306/cardio_data/cardio_engineered_clean.csv

upload: ./cardio_model_monitoring.ipynb to s3://sagemaker-us-east-1-531690656306/cardio_project/cardio_model_monitoring.ipynb
upload: ./cardio_engineered_clean.csv to s3://sagemaker-us-east-1-531690656306/cardio_data/cardio_engineered_clean.csv




## For Reference Only
### <u>File Locations Summary for Model Monitoring Setup</u>

#### Trained Model Artifact (used for deployment & endpoint)
`s3://sagemaker-us-east-1-531690656306/model/logistic_model.tar.gz`

#### Training Dataset (used for baseline generation)
`s3://sagemaker-us-east-1-531690656306/cardio_data/cardio_train.csv`

#### Engineered Dataset (cleaned and engineered)
`s3://sagemaker-us-east-1-531690656306/cardio_data/cardio_engineered_clean.csv`

#### Baseline Results Output Folder (generated by model monitor baseline job)
`s3://sagemaker-us-east-1-531690656306/cardio_data/baseline-results/`

#### Inside baseline-results:
- `statistics.json`  
  `s3://sagemaker-us-east-1-531690656306/cardio_data/baseline-results/statistics.json`

- `constraints.json`  
  `s3://sagemaker-us-east-1-531690656306/cardio_data/baseline-results/constraints.json`

#### Data Capture Location (request/response payloads captured from endpoint)
`s3://sagemaker-us-east-1-531690656306/data-capture/`